In [1]:
#! pip install cherrypy

In [2]:
#! pip install markupsafe==2.0.1

In [3]:
#! pip install itsdangerous==2.0.1

In [4]:
#! pip install -U Werkzeug==0.16.0

In [5]:
import pandas as pd
import numpy as np
from app_function import view_classifica, view_player_page, view_role_explorer, view_team_builder, view_builded_team
from team_builder import build_team

In [6]:
actual_show_fav = True
actual_show_not_fav = True

In [7]:
actual_role_selected = 'P'
actual_min_pres = 0
actual_team_selected = ' '

In [8]:
actual_team_list = []
actual_modulo_scelto = '3-4-3'

In [9]:
player_info = pd.read_csv('../dataset/stat_info^(Stagione 2021_22).csv',index_col=[0])
player_hist = pd.read_csv('../dataset/player_history^(Stagione 2021_22).csv',index_col=[0])

player_hist.Voto = player_hist.Voto.astype(float)
player_hist.FV = player_hist.FV.astype(float)
player_hist.Bonus = player_hist.Bonus.astype(float)
player_hist.Malus = player_hist.Malus.astype(float)

all_names = player_info.Nome.unique().tolist()
all_teams = player_info.Squadra.unique().tolist()

classifica = pd.read_csv('../dataset/final_classifica^(Stagione 2021_22).csv',index_col=[0])

In [ ]:
from flask import Flask, render_template,request, redirect, url_for

app = Flask(__name__)

@app.route("/")
def home_page():
    return render_template('home_page.html')

@app.route("/player-page")
def player_page(nome=all_names[0]):
    return view_player_page(player_info,player_hist,all_names,nome)
    
@app.route('/player-page', methods=['POST'])
def handle_name():
    nome_selected= request.form.get('nome_selezionato')
    return view_player_page(player_info,player_hist,all_names,nome_selected)

@app.route('/classifica', methods=("POST", "GET"))
def classfica_page():
    return view_classifica(classifica)

@app.route('/role-explorer')
def role_page():
    fav_file = open("../tmp/favourite.txt", "r")
    return view_role_explorer(player_info, all_teams, fav_file = fav_file )

@app.route('/role-explorer', methods=['POST'])
def handle_role_explorer():
    
    global actual_show_fav
    global actual_show_not_fav
    
    if 'checked_fav' in request.form or  'checked_not_fav' in request.form:
        
        if 'checked_fav' in request.form:
            show_fav = True
        else:
            show_fav = False

        if 'checked_not_fav' in request.form:
            show_not_fav = True
        else:
            show_not_fav = False

        actual_show_fav = show_fav
        actual_show_not_fav = show_not_fav

    if 'add_to_fav' in request.form:
        
        name_for_fav = request.form.get('add_to_fav')    
        file_in = open('../tmp/favourite.txt',"r")

        names_favourite = file_in.read().splitlines()
        if '' in names_favourite: 
            names_favourite.remove('')
        file_in.close()
        
        if name_for_fav in names_favourite:
            file_write = open('../tmp/favourite.txt',"w")
            names_favourite.remove(name_for_fav)
            for names in names_favourite:
                if names != '':
                    file_write.write(names + '\n')
            file_write.close()
        else:
            file_append = open('../tmp/favourite.txt',"a")
            file_append.write(name_for_fav +'\n')
            file_append.close()
        
    else:
        
        global actual_min_pres
        min_pres = request.form.get('min_pres')
        actual_min_pres = min_pres
        global actual_role_selected
        role_selected = request.form.get('role_selected')
        actual_role_selected = role_selected
        global actual_team_selected
        team_selected = request.form.get('team_selected')
        actual_team_selected = team_selected
        
    fav_file = open("../tmp/favourite.txt", "r")
    
    return view_role_explorer(player_info, all_teams, fav_file = fav_file,
                              role = actual_role_selected, 
                              filter_presence = actual_min_pres, 
                              team_selected = actual_team_selected,
                              show_fav = actual_show_fav,
                              show_not_fav = actual_show_not_fav)

@app.route('/team-builder')
def team_builder():
    return view_team_builder(player_info, actual_modulo_scelto)
            
@app.route('/team-builder', methods=['POST'])
def handle_team_builder():
    team_list = []
    if 'scegli_modulo' in request.form:
        global actual_modulo_scelto
        modulo_scelto = request.form.get('modulo_scelto')
        actual_modulo_scelto = modulo_scelto
        return view_team_builder(player_info, actual_modulo_scelto)
    
    if 'build_team' in request.form:
        for i in range(1,12):
            team_list.append(request.form.get('nome_'+str(i)))
        return view_builded_team(team_list, player_info, player_hist)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2022 15:30:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/styles/homepage_css.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/styles/general.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/img/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/img/player_page.PNG HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/img/classifica.PNG HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/img/role_explorer.PNG HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /static/img/team_builder.PNG HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Jul/2022 15:30:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:30:52] "GET /static/styles/homepage_css.css HTTP/1.1" 200 -

127.0.0.1 - - [29/Jul/2022 15:34:00] "GET /player-page HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:03] "GET /role-explorer HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:03] "GET /player-page HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:04] "GET /team-builder HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:09] "GET /classifica HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:11] "GET /role-explorer HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:13] "GET /team-builder HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:24] "GET /team-builder HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:26] "GET /classifica HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:28] "GET /role-explorer HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 15:34:29] "GET /player-page HTTP/1.1" 200 -
